In [1]:
import adlfs

In [2]:
# use read only token
sas_token = 'se=2023-03-13T02%3A45Z&sp=racwdl&sv=2018-11-09&sr=c&skoid=b4d39ccb-ca9c-4e9d-a183-5e0f3ba3e609&sktid=f6b6dd5b-f02f-441a-99a0-162ac5060bd2&skt=2023-03-06T20%3A38%3A54Z&ske=2023-03-13T02%3A45%3A00Z&sks=b&skv=2018-11-09&sig=Nl9p59uFIaJRJgtbHhnVBgYgwh1MnZ/fcJrttie9gAM%3D'
tile_names = [tile[-5:] for tile in adlfs.AzureBlobFileSystem(account_name="snowmelt", credential=sas_token, anon=False).ls('snowmelt/eric/MGRS/')]

In [3]:
years = ['allyears_median','allyears_std','allyears_corr_strength','2015_median','2016_median','2017_median','2018_median','2019_median','2020_median','2021_median','2022_median']
UTM_zone = [10,11,12,13]
resolution = 80

In [4]:
for year in years:
    for zone in UTM_zone:
        f = open(f'vrts/filenames/files_{year}_utm{zone}.txt','w')
        tile_names_zone = [s for s in tile_names if f'{zone}' in s]
        for tile in tile_names_zone:
            tif = f"https://snowmelt.blob.core.windows.net/snowmelt/eric/MGRS/{tile}/runoff_onset_{tile}_{year}_{resolution}m.tif"
            f.write(f'{tif}\n')
        f.close()

In [5]:
for year in years:
    if (year == 'allyears_std') | (year == 'allyears_corr_strength'):
        nodata = "nan"
    else:
        nodata = '-32768'
    for zone in UTM_zone:
        !gdalbuildvrt -input_file_list $f'vrts/filenames/files_{year}_utm{zone}.txt' $f'vrts/vrt_{year}_utm{zone}.vrt' -vrtnodata $nodata

0...10...20...30...40ERROR 1: OpenSSL SSL_read: Connection reset by peer, errno 104
Warning 1: Can't open https://snowmelt.blob.core.windows.net/snowmelt/eric/MGRS/10TFL/runoff_onset_10TFL_allyears_median_80m.tif. Skipping it
...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90ERROR 1: OpenSSL SSL_read: Connection reset by peer, errno 104
Warning 1: Can't open https://snowmelt.blob.core.windows.net/snowmelt/eric/MGRS/13UDQ/runoff_onset_13UDQ_allyears_median_80m.tif. Skipping it
...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40..ERROR 1: OpenSSL SSL_read: Connection reset by peer, errno 104
Warning 1: Can't open https://snowmelt.blob.core.windows.net/snowmelt/eric/MGRS/11TMJ/runoff_onset_11TMJ_allyears_std_80m.tif. Skipping it
.50...60...70...80...90...100 - done.
0...10...20...30...40.